In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
pd.set_option('display.max_columns', 100)

In [5]:
nst_to_sched = { 'Anaheim Ducks': 'ANA', 
                      'Arizona Coyotes' : 'ARI', #ARI after 2013-14 season
                      'Boston Bruins': 'BOS', 
                      'Buffalo Sabres':'BUF',
                      'Calgary Flames': 'CGY', 
                      'Carolina Hurricanes': 'CAR', 
                      'Chicago Blackhawks': 'CHI', 
                      'Colorado Avalanche': 'COL',
                     'Columbus Blue Jackets': 'CBJ',
                     'Dallas Stars': 'DAL',
                     'Detroit Red Wings': 'DET',
                     'Edmonton Oilers': 'EDM',
                     'Florida Panthers': 'FLA',
                     'Los Angeles Kings': 'L.A.',
                     'Minnesota Wild': 'MIN',
                     'Montreal Canadiens': 'MTL',
                     'Nashville Predators': 'NSH',
                     'New Jersey Devils': 'N.J.',
                     'New York Islanders ': 'NYI',
                     'New York Rangers ': 'NYR',
                     'Ottawa Senators': 'OTT',
                     'Philadelphia Flyers': 'PHI',
                     'Pittsburgh Penguins': 'PIT',
                     'San Jose Sharks': 'S.J.',
                     'St. Louis Blues': 'STL',
                     'Tampa Bay Lightning': 'T.B.',
                     'Toronto Maple Leafs': 'TOR',
                     'Vancouver Canucks': 'VAN',
                      'Vegas Golden Knights':'VGK',
                     'Washington Capitals': 'WSH',
                     'Winnipeg Jets': 'WPG'}

In [6]:
goalie_table_teams = { 'ANA': 'ANA', 
                      'ARI' : 'ARI',
                      'BOS': 'BOS', 
                      'BUF':'BUF',
                      'CGY': 'CGY', 
                      'CAR': 'CAR', 
                      'CHI': 'CHI', 
                      'COL': 'COL',
                     'CBJ': 'CBJ',
                     'DAL': 'DAL',
                     'DET': 'DET',
                     'EDM': 'EDM',
                     'FLA': 'FLO',
                     'L.A': 'LOS',
                     'MIN': 'MIN',
                     'MTL': 'MON',
                     'NSH': 'NSH',
                     'N.J': 'NJD',
                     'NYI': 'NYI',
                     'NYR': 'NYR',
                     'OTT': 'OTT',
                     'PHI': 'PHI',
                     'PIT': 'PIT',
                     'S.J': 'SJS',
                     'STL': 'STL',
                     'T.B': 'TBL',
                     'TOR': 'TOR',
                     'VAN': 'VAN',
                     'WSH': 'WSH',
                     'WPG': 'WPG',
                     'VGK':'VGK'}

In [2]:
def get_and_format_nst_team_stats(date, sit, rate):
    url = 'https://www.naturalstattrick.com/games.php?fromseason={}&thruseason={}&stype=2&sit={}&loc=B&team=All&rate={}'.format(
        season,
        season,
        sit,
        rate)
    df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
    df.reset_index(inplace = True)
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Game_Number'] = df.groupby('Team').cumcount() + 1
    #rename Team_Date to team key or something like that
    df = df.replace({'Team': nst_to_sched})

    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    return df

In [3]:
def merge_team_stats(primary_df, pp_df, pk_df):
    primary_df = primary_df.merge(pk_df[['Team_Key', 'TOI', 'xGA']], on = 'Team_Key', how = 'left', suffixes = ('','_pk') )
    primary_df = primary_df.merge(pp_df[['Team_Key', 'TOI', 'xGF']], on = 'Team_Key', how = 'left', suffixes = ('','_pp') )
    return primary_df

In [4]:
def calculate_team_features(df, rolling_games = 20):
    df['sum_rolling20_TOI_5v5'] = df.groupby('Team')['TOI'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_GF_5v5'] = df.groupby('Team')['GF'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_GA_5v5'] = df.groupby('Team')['GA'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_xGF_5v5'] = df.groupby('Team')['xGF'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_xGA_5v5'] = df.groupby('Team')['xGA'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_SF_5v5'] = df.groupby('Team')['SF'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['last_20_FF%_5v5'] = df['sum_rolling20_FF_5v5']*100/ (df['sum_rolling20_FF_5v5']+df['sum_rolling20_FA_5v5'])
    df['last_20_GF%_5v5'] = df['sum_rolling20_GF_5v5']*100/ (df['sum_rolling20_GF_5v5']+df['sum_rolling20_GA_5v5'])
    df['last_20_xGF%_5v5'] = df['sum_rolling20_xGF_5v5']*100/ (df['sum_rolling20_xGF_5v5']+df['sum_rolling20_xGA_5v5'])
    df['last_20_SH%'] = df['sum_rolling20_GF_5v5']*100 / df['sum_rolling20_SF_5v5']
    
    
    #fix NaNs in pp and pk features
    df['TOI_pp'] = np.where(df['TOI_pp'].isna(), 0, df['TOI_pp'])
    df['TOI_pk'] = np.where(df['TOI_pk'].isna(), 0, df['TOI_pk'])
    df['xGF_pp'] = np.where(df['xGF_pp'].isna(), 0, df['xGF_pp'])
    df['xGA_pk'] = np.where(df['xGA_pk'].isna(), 0, df['xGA_pk'])
    
    #pp features
    df['sum_rolling20_TOI_pp'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_xGF_pp'] = df.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['last20_pp_TOI_per_game'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(20, 20).mean().shift())
    df['last20_xGF_per_min_pp'] = df['sum_rolling20_xGF_pp'] / df['sum_rolling20_TOI_pp'] 
    
    #pk features
    df['sum_rolling20_TOI_pk'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['sum_rolling20_xGA_pk'] = df.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(20, 20).sum().shift())
    df['last20_pk_TOI_per_game'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(20, 20).mean().shift())
    df['last20_xGA_per_min_pk'] = df['sum_rolling20_xGA_pk'] / df['sum_rolling20_TOI_pk'] 
    
    #to get back to back category
    df['Last_Game_Date'] = df.groupby('Team')['Date'].shift()
    df['Days_Since_Last_Game'] = df['Date'] - df['Last_Game_Date']
    df['B2B'] = np.where(df['Days_Since_Last_Game'] == '1 days', 1, 0)
    
    return df

In [8]:
#get starters
def get_starters(year):
    counter = 0
    for k,v in goalie_table_teams.items():
        print(k)
        starter_url = 'http://hockeygoalies.org/bio/nhl/logs/{v}{year}.html'.format(v, year)
        goalies = pd.read_html(starter_url)[0]
        goalies.replace(to_replace=['(BU)', np.NaN], value = 'DNP', inplace = True)
        goalies.drop(columns = ['DEC'], inplace = True)
        goalies.drop(index  = goalies.iloc[-1].name, inplace = True)
        goalies['starter'] = 'placeholder'

        starter = []
        for i, row in goalies.iterrows():
            for n in range(len(row)):
                if row[n][0] == 'W' or row[n][0] == 'L':
                    starter.append(goalies.columns[n])

        goalies['starter'] = starter
        goalies['Team'] = k
        goalies['DATE'] = pd.to_datetime(goalies['DATE'])
        goalies['DATE'] = pd.to_datetime(goalies['DATE'])
        goalies['Team_Key'] = goalies['Team'].astype(str)+'_'+goalies['DATE'].astype(str)
        columns = ['Team','DATE', 'OPPONENT', 'starter', 'Team_Key']
        if counter == 0:
            master = goalies
        if counter != 0:
            master = pd.concat( [master[columns], goalies[columns]])
        counter +=1
                

In [ ]:
#'2017-10-04' to '2018-04-08'
def get_game_results(season_start, season_end):
    sched_df = hockey_scraper.scrape_schedule(season_start, season_end)
    sched_df['Home_Team_Won'] = np.where(sched_df['home_score'] > sched_df['away_score'], 1, 0)
    sched_df['Home_Team_Key'] = sched_df['home_team'].astype(str)+'_'+sched_df['date'].astype(str)
    sched_df['Away_Team_Key'] = sched_df['away_team'].astype(str)+'_'+sched_df['date'].astype(str)

In [ ]:
def merge_starters_and_features(game_results_df, starters_df, features_df):
    df = game_results_df.merge(starters_df[['Team_Key', 'starter']], left_on = 'Home_Team_Key', right_on = 'Team_Key', how = 'left').rename(columns ={'starter':'home_starter'}).drop(columns = 'Team_Key')
    df = df.merge(game_results_df[['Team_Key', 'starter']], left_on = 'Away_Team_Key', right_on = 'Team_Key', how = 'left').rename(columns ={'starter':'away_starter'}).drop(columns = 'Team_Key')
    test = sched_df.merge(tf[feature_columns].add_prefix('home_'), left_on = 'Home_Team_Key', right_on = 'home_Team_Key', how = 'left')
test = test.merge(tf[feature_columns].add_prefix('away_'), left_on = 'Away_Team_Key', right_on = 'away_Team_Key', how = 'left')

### 2017-2018 Season